# Final Scoring in the Competition and Ranking!

__Submission__

![title](submission_score.png)

__Leaderboard__

![title](leaderboard.png)

__Summary of Improvements__
* Since we are dealing with image data, I would think it would be best to use a few _convolutional_ and _pooling_ layers for this case
* Added a _Dropout layer_: To prevent overfitting
* Adjust _Neurons_ per layer
* Added _Learning Rate Scheduling_: Piecewise Constant Scheduling 
* Added _Batch Normalization_: To standardizes the inputs and stabilize the learning process
* Used activation function (_Elu_) and initialization of weights (_lecun_normal_)
* Added more layers and neurons
* Used a different type of optimizer (_Adam_)
* Added a _gradient clipping_: To prevent exploding gradients

__Imports__

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer

__Dataset__

In [6]:
digit = pd.read_csv("data/train.csv")
digit_test = pd.read_csv("data/test.csv")

__Training Dataset View__

In [7]:
digit.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__Test Dataset View__

In [8]:
digit_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


__Get only values__

In [9]:
x = digit.drop('label', axis = 1).values
y = digit.label.values

In [10]:
x.shape

(42000, 784)

In [11]:
y.shape

(42000,)

__Re-scaling__

In [12]:
x = x/255.0
test_df = digit_test.values/255.0

__Reshaping__

In [14]:
x_train = x.reshape(x.shape[0], 28, 28,1)
test_df = test_df.reshape(digit_test.shape[0], 28, 28,1)
y_train = y.reshape(-1,1)

__Splitting Data__

In [15]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size = 0.20, 
                                                      random_state = 141)

__Clear the Backend__

In [18]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

__Piecewise Constant Scheduling__
* Added an additional learning constant
* A dynamic learning process

In [19]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 10:
        return 0.005
    elif epoch < 15:
        return 0.003
    elif epoch < 20:
        return 0.001
    else:
        return 0.0001

In [20]:
def piecewise_constant(boundaries, values):
    boundaries = np.array([0] + boundaries)
    values = np.array(values)
    def piecewise_constant_fn(epoch):
        return values[np.argmax(boundaries > epoch) - 1]
    return piecewise_constant_fn
piecewise_constant_fn = piecewise_constant([5,10,15,20,25], [0.01, 0.005,0.003,0.001,0.0001])

In [21]:
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

__Model Creation__

In [36]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=64, kernel_size=(5, 5),padding="valid" ,
                              activation='elu',kernel_initializer="lecun_normal", input_shape=(28,28,1)))
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters=32, kernel_size=(5, 5),padding="valid", 
                              activation='elu',kernel_initializer="lecun_normal"))
model.add(keras.layers.AveragePooling2D())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.GaussianDropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=128, activation='selu',kernel_initializer="lecun_normal"))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(units=64, activation='selu',kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(units=10, activation = 'softmax'))

In [37]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 24, 24, 64)        1664      
                                                                 
 average_pooling2d_4 (Averag  (None, 12, 12, 64)       0         
 ePooling2D)                                                     
                                                                 
 batch_normalization_14 (Bat  (None, 12, 12, 64)       256       
 chNormalization)                                                
                                                                 
 conv2d_7 (Conv2D)           (None, 8, 8, 32)          51232     
                                                                 
 average_pooling2d_5 (Averag  (None, 4, 4, 32)         0         
 ePooling2D)                                                     
                                                      

__Model Compile__

In [38]:
model.compile(loss="sparse_categorical_crossentropy",optimizer=keras.optimizers.Adam(clipvalue=1.0,clipnorm=1.0),metrics=["accuracy"])

__Model Fitting__

In [39]:
history = model.fit(x_train,y_train,epochs=25,validation_data=(x_valid,y_valid),callbacks=[lr_scheduler])

Epoch 1/25


2022-02-26 15:25:47.913170: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1049/1050 [============================>.] - ETA: 0s - loss: 0.2338 - accuracy: 0.9304

2022-02-26 15:26:21.398391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1050/1050 [==============================] - 37s 34ms/step - loss: 0.2337 - accuracy: 0.9304 - val_loss: 0.0989 - val_accuracy: 0.9731 - lr: 0.0100
Epoch 2/25
1050/1050 [==============================] - 33s 32ms/step - loss: 0.1190 - accuracy: 0.9651 - val_loss: 0.0793 - val_accuracy: 0.9771 - lr: 0.0100
Epoch 3/25
1050/1050 [==============================] - 33s 32ms/step - loss: 0.1025 - accuracy: 0.9709 - val_loss: 0.2088 - val_accuracy: 0.9464 - lr: 0.0100
Epoch 4/25
1050/1050 [==============================] - 34s 33ms/step - loss: 0.0933 - accuracy: 0.9731 - val_loss: 0.0791 - val_accuracy: 0.9789 - lr: 0.0100
Epoch 5/25
1050/1050 [==============================] - 33s 32ms/step - loss: 0.0853 - accuracy: 0.9770 - val_loss: 0.0539 - val_accuracy: 0.9880 - lr: 0.0100
Epoch 6/25
1050/1050 [==============================] - 34s 32ms/step - loss: 0.0467 - accuracy: 0.9856 - val_loss: 0.0412 - val_accuracy: 0.9890 - lr: 0.0050
Epoch 7/25
1050/1050 [==============================] - 3

__Save the Model__
* I already saved this model to local and cloud

In [112]:
#model.save("Digit_Recog_Mod_2.h5")

In [113]:
ml = keras.models.load_model("Digit_Recog_Mod_2.h5")

__Testing Model Against Test Data__

In [114]:
pred = ml.predict(test_df)

2022-02-26 17:12:07.516663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


__Get the index of the max value in the array__

In [117]:
prediction = [np.argmax(i) for i in pred]

[2,
 0,
 9,
 0,
 3,
 7,
 0,
 3,
 0,
 3,
 5,
 7,
 4,
 0,
 4,
 3,
 3,
 1,
 9,
 0,
 9,
 1,
 1,
 5,
 7,
 4,
 2,
 7,
 4,
 7,
 7,
 5,
 4,
 2,
 6,
 2,
 5,
 5,
 1,
 6,
 7,
 7,
 4,
 9,
 8,
 7,
 8,
 2,
 6,
 7,
 6,
 8,
 8,
 3,
 8,
 2,
 1,
 2,
 2,
 0,
 4,
 1,
 7,
 0,
 0,
 0,
 1,
 9,
 0,
 1,
 6,
 5,
 8,
 8,
 2,
 8,
 9,
 9,
 2,
 3,
 5,
 4,
 1,
 0,
 9,
 2,
 4,
 3,
 6,
 7,
 2,
 0,
 6,
 6,
 1,
 4,
 3,
 9,
 7,
 4,
 0,
 9,
 2,
 0,
 7,
 3,
 0,
 5,
 0,
 8,
 0,
 0,
 4,
 7,
 1,
 7,
 1,
 1,
 3,
 3,
 3,
 7,
 2,
 8,
 6,
 3,
 8,
 7,
 7,
 4,
 3,
 5,
 6,
 0,
 0,
 0,
 3,
 1,
 3,
 6,
 4,
 3,
 4,
 5,
 5,
 8,
 7,
 7,
 2,
 8,
 4,
 3,
 5,
 6,
 5,
 3,
 7,
 5,
 7,
 8,
 3,
 0,
 4,
 5,
 1,
 2,
 7,
 6,
 3,
 0,
 2,
 7,
 8,
 6,
 1,
 3,
 7,
 4,
 1,
 2,
 4,
 8,
 5,
 2,
 4,
 9,
 2,
 1,
 6,
 0,
 6,
 1,
 4,
 9,
 6,
 0,
 9,
 7,
 6,
 9,
 1,
 9,
 0,
 9,
 9,
 0,
 8,
 4,
 6,
 2,
 0,
 9,
 3,
 6,
 3,
 2,
 1,
 6,
 3,
 4,
 2,
 3,
 1,
 2,
 2,
 0,
 4,
 6,
 1,
 0,
 0,
 4,
 9,
 1,
 7,
 3,
 2,
 3,
 8,
 6,
 8,
 6,
 2,
 8,
 5,
 5,
 4,
 8,
 3,
 5,


In [102]:
img_id = list(range(1,28001))
img_id = np.array(img_id)

In [104]:
img_id.shape

(28000,)

__Final Dataframe of Predictions__
* Has to be in this format for the competition

In [105]:
df = pd.DataFrame({"ImageId":img_id,"Label":prediction})
df.head()

In [106]:
df.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [107]:
df.tail()

,ImageId,Label
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
27999,28000,2


__Download to csv format__

In [108]:
df.to_csv("Michael_Woo_Predictions_Digit_Recog_2.csv",index=False)